In [1]:
import numpy as np 
from scipy.integrate import odeint
import os, sys, warnings
from pathlib import Path
from os.path import dirname, realpath
script_dir = Path(dirname(realpath('.')))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
import utility as ut
import surrogate_tf as srnn
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
warnings.filterwarnings('ignore')

In [2]:
L63_data_path = '../data/L63-trajectories'
N = 20000
train = np.load(f'{L63_data_path}/train.npy')[:, :N].astype(np.float32)
test = np.load(f'{L63_data_path}/test.npy')[:, :, :1000].astype(np.float32)
model = srnn.SurrogateModel_NN(3, 300)

2024-01-11 08:29:18.984757: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-01-11 08:29:18.984776: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-01-11 08:29:18.984781: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-01-11 08:29:18.984809: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-11 08:29:18.984822: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
model.learn(train, epochs=2000, learning_rate=1e-4)

2024-01-11 08:29:19.192583: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-01-11 08:29:19.201100: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


epoch: 0    loss: 240.755234      time elapsed=0.2732
epoch: 100    loss: 18.515656      time elapsed=1.0103
epoch: 200    loss: 9.359363      time elapsed=1.7444
epoch: 300    loss: 4.688671      time elapsed=2.4837
epoch: 400    loss: 2.662088      time elapsed=3.3394
epoch: 500    loss: 1.786272      time elapsed=4.0847
epoch: 600    loss: 1.347949      time elapsed=4.8755
epoch: 700    loss: 1.075942      time elapsed=5.6640
epoch: 800    loss: 0.875607      time elapsed=6.4052
epoch: 900    loss: 0.723128      time elapsed=7.1682
epoch: 1000    loss: 0.605519      time elapsed=7.9731
epoch: 1100    loss: 0.511454      time elapsed=8.7150
epoch: 1200    loss: 0.434658      time elapsed=9.4527
epoch: 1300    loss: 0.371730      time elapsed=10.2188
epoch: 1400    loss: 0.319781      time elapsed=11.0244
epoch: 1500    loss: 0.276624      time elapsed=11.7955
epoch: 1600    loss: 0.240461      time elapsed=12.5918
epoch: 1700    loss: 0.209991      time elapsed=13.3372
epoch: 1800   

In [4]:
tau_f_rmse, tau_f_se, rmse, se = model.compute_tau_f(test[:500, :, :200], error_threshold=0.05)

Working with index = 0


NotImplementedError: in user code:

    File "/Users/pman0581/Documents/GitHub/random-feature-map/modules/surrogate_tf.py", line 67, in multistep_forecast  *
        trajectory[:, 0] = u

    NotImplementedError: Cannot convert a symbolic tf.Tensor (u:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported.


In [ ]:
plt.hist(tau_f_se, density=True)
plt.xlabel(r'$\tau_f$ (NN)')
plt.savefig('../data/plots/tau_f_NN.png')
# plt.title(f'training_data_size={N}')

In [ ]:
tau_f_rmse, tau_f_se, rmse, se = model.compute_tau_f(train.T.reshape(-1, 3, 800))

plt.hist(tau_f_se)
plt.xlabel(r'$\tau_f$ (NN), for train data')
plt.title(f'training_data_size={N}')

In [ ]:
fig, N, dt = plt.figure(figsize=(8, 8)), 500, 0.02
t = np.arange(0., N*dt, dt) / (1/0.91)
axs = [fig.add_subplot(311), fig.add_subplot(312), fig.add_subplot(313)]
dims = ['x', 'y', 'z']
u = np.load('{}/validation.npy'.format(L63_data_path))

idx = [[4, 14, 18], [13, 17, 46], [13, 2, 31], ]
labels = ['good', 'medium', 'bad']
predicted = model.multistep_forecast(u[:, 0], N)
for i, ax in enumerate(axs):
    ax.plot(t, u[i, :N], label='truth')
    ax.plot(t, predicted[i, :N], label='network')
    ax.legend(loc='upper right')
    if i == len(dims)-1:
        ax.set_xlabel('t')
    ax.set_ylabel(dims[i])

In [ ]:
test[:500].shape

In [ ]:
model.multistep_forecast(np.random.normal(size=(1, 3)).astype(np.float32), 10).T

In [8]:
df = pd.DataFrame({'iteration': [], 'loss': [], 'runtime': [], 'hardware': []})
df.to_csv('empty.csv', index=None)

In [5]:
df

""
